# Run Manual Image Labelling, if needed

In [5]:
import os
import Configuration.Config as config
LABELIMG_PATH = config.paths["LABELIMG_PATH"]
if os.name == 'posix':
    !cd {LABELIMG_PATH} && pyrcc5 -o libs/resources.py resources.qrc && !make qt5py3
if os.name =='nt':
    !cd {LABELIMG_PATH} && pyrcc5 -o libs/resources.py resources.qrc
!cd {LABELIMG_PATH} && python labelImg.py

^C


# Dependency script

In [ ]:
import PackageInstaller
PackageInstaller.installDependencies()

#  LabelMap Creation

In [ ]:
import TensorTrain
TensorTrain.Train()

# Installation of TensorFlow ObjectDetection

In [5]:
import wget
import Configuration.Config as config
import os
if os.name=='posix':
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .

if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    protocPath = config.paths['PROTOC_PATH']
    !move protoc-3.15.6-win64.zip {protocPath}
    !cd {protocPath} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(protocPath, 'bin'))
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e .

        1 file(s) moved.


The system cannot find the path specified.
The system cannot find the path specified.


# Verification of objectDetection script

In [7]:
import Configuration.Config as config
import os

apiModelPath = config.paths['APIMODEL_PATH']
VERIFICATION_SCRIPT = os.path.join(apiModelPath, 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

Running tests under Python 3.9.7: C:\ProgramData\Anaconda3\envs\tensorFlowObjectDetection\python.exe
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-04-07 12:18:45.964263: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-07 12:18:47.239515: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5470 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:09:00.0, compute capability: 8.6
C:\ProgramData\Anaconda3\envs\tensorFlowObjectDetection\lib\site-packages\object_detection-0.1-py3.9.egg\object_detection\builders\model_builder.py:1100: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead


# PreTrained model installing

In [ ]:
import wget
import os
import Configuration.Config as config

PRETRAINED_MODEL_PATH = config.paths['PRETRAINED_MODEL_PATH']
PRETRAINED_MODEL_URL = config.PRETRAINED_MODEL_URL
PRETRAINED_MODEL_NAME = config.PRETRAINED_MODEL_NAME
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {pretrainedModelUrlPath}
    !cd {pretrainedModelUrlPath} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    !wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {pretrainedModelUrlPath}
    !cd {PRETRAINED_MODEL_PATH} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

#  Creating Tensorflow records

In [ ]:
import os
import Configuration.Config as config
TF_RECORD_SCRIPT = config.files['TF_RECORD_SCRIPT']
IMAGE_PATH = config.paths['IMAGE_PATH']
LABELMAP = config.files['LABELMAP']
ANNOTATION_PATH = config.paths['ANNOTATION_PATH']
!python {TF_RECORD_SCRIPT} -x {os.path.join(IMAGE_PATH, 'train')} -l {LABELMAP} -o {os.path.join(ANNOTATION_PATH, 'train.record')}
!python {TF_RECORD_SCRIPT} -x {os.path.join(IMAGE_PATH, 'test')} -l {LABELMAP} -o {os.path.join(ANNOTATION_PATH, 'test.record')}

# Copying Model Config to Training Folder

In [ ]:
import os
import Configuration.Config as config
PRETRAINED_MODEL_PATH = config.paths['PRETRAINED_MODEL_PATH']
PRETRAINED_MODEL_NAME = config.PRETRAINED_MODEL_NAME
CHECKPOINT_PATH = config.paths['CHECKPOINT_PATH']

if os.name =='posix':
    !cp {os.path.join(PRETRAINED_MODEL_PATH, PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(CHECKPOINT_PATH)}
if os.name == 'nt':
    !copy {os.path.join(PRETRAINED_MODEL_PATH, PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(CHECKPOINT_PATH)}


# Updating Config For Transfer Learning

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
import Configuration.Config as config
import TensorTrain

PIPELINE_CONFIG = config.files['PIPELINE_CONFIG']
labels = TensorTrain.GetLabels()
PRETRAINED_MODEL_PATH = config.paths['PRETRAINED_MODEL_PATH']
PRETRAINED_MODEL_NAME = config.PRETRAINED_MODEL_NAME
LABELMAP = config.files['LABELMAP']
ANNOTATION_PATH = config.paths['ANNOTATION_PATH']
PIPELINE_CONFIG = config.files['PIPELINE_CONFIG']

config = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG)
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(PIPELINE_CONFIG, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(PRETRAINED_MODEL_PATH, PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = LABELMAP
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(ANNOTATION_PATH, 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = LABELMAP
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(ANNOTATION_PATH, 'test.record')]

config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(PIPELINE_CONFIG, "wb") as f:
    f.write(config_text)
config = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG)
print(config)

# 6. Train the model

In [2]:
import os
import Configuration.Config as config
import object_detection

APIMODEL_PATH = config.paths['APIMODEL_PATH']
TRAINING_SCRIPT = os.path.join(APIMODEL_PATH, 'research', 'object_detection', 'model_main_tf2.py')
CHECKPOINT_PATH = config.paths['CHECKPOINT_PATH']
PIPELINE_CONFIG = config.files['PIPELINE_CONFIG']
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, CHECKPOINT_PATH,PIPELINE_CONFIG)
print(command)
# For output it's better to launch the command in anaconda command line

python C:\Users\User\PycharmProjects\tensorFlowObjectDetection\TFRecognition\Tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=C:\Users\User\PycharmProjects\tensorFlowObjectDetection\TFRecognition\Tensorflow\workspace\models\my_ssd_mobnet_1000_pipeline_2k_run --pipeline_config_path=C:\Users\User\PycharmProjects\tensorFlowObjectDetection\TFRecognition\Tensorflow\workspace\models\my_ssd_mobnet_1000_pipeline_2k_run\pipeline.config --num_train_steps=2000


In [3]:
#running the learning model in jupyter notebook
!{command}


2022-04-07 14:46:22.898209: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-07 14:46:23.621833: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5470 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:09:00.0, compute capability: 8.6
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0407 14:46:23.794654 22656 mirrored_strategy.py:369] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 2000
I0407 14:46:23.801661 22656 config_util.py:552] Maybe overwriting train_steps: 2000
INFO:tensorflow:Ma

# Model Evaluation


In [1]:
import os
import Configuration.Config as config

TRAINING_SCRIPT = os.path.join(APIMODEL_PATH, 'research', 'object_detection', 'model_main_tf2.py')
CHECKPOINT_PATH = config.paths['CHECKPOINT_PATH']
PIPELINE_CONFIG = config.files['PIPELINE_CONFIG']

command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, CHECKPOINT_PATH,PIPELINE_CONFIG, CHECKPOINT_PATH)
print(command)
# Better to execute the command in anaconda

NameError: name 'APIMODEL_PATH' is not defined

In [ ]:
!{command}

